In [1]:
import re
import os
import pandas as pd
import nltk
import textblob
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
pip install textblob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# loading the from Google drive
# This dataset is about based on twitter tweet get to know whether person in depression mood or not

data= pd.read_csv(r"/content/sentiment_tweets3.csv")
print(data.shape)
data.head(3)

(10314, 3)


,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0


In [13]:
col= {"message to examine": "text", "label (depression result)":"label"}
data= data.rename(columns=col)

In [14]:
# loading from local machine

# data= pd.read_csv(r"F:\Generative AI\dataset\twitter customer support\twcs.csv")
# print(data.shape)
# data.head(3)

In [15]:
data.columns

Index(['Index', 'text', 'label'], dtype='object')

In [16]:
data.shape

(10314, 3)

In [17]:
# Label column indicates Does the person have depression? 0 stands for NO and 1 stands for YES

data['label'].value_counts()

0    8000
1    2314
Name: label, dtype: int64

In [18]:
data.head(5)

,Index,text,label
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


### Removing the HTML tags

In [19]:
# removing the HTML tags in dataframe

def remove_html_tags(text):
    pattern= re.compile('<.*?>')
    return pattern.sub("", text)

data['text']= data['text'].apply(remove_html_tags)

### Removing the urls in data

In [20]:
# removing the urls in text column

def remove_urls(text):
    pattern= re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub("", text)

data['text']= data['text'].apply(remove_urls)

### To Remove the abbrivation words like ASAP, PFA.

In [21]:
chat_words={
"AFAIK":"As Far As I Know", "AFK":"Away From Keyboard",
"ASAP":"As Soon As Possible", "ATK":"At The Keyboard",
"ATM":"At The Moment", "A3":"Anytime, Anywhere, Anyplace",
"BAK":"Back At Keyboard", "BBL":"Be Back Later",
"BBS":"Be Back Soon", "BFN":"Bye For Now",
"B4N":"Bye For Now", "BRB":"Be Right Back",
"BRT":"Be Right There", "BTW":"By The Way",
"B4":"Before", "B4N":"Bye For Now",
"CU":"See You", "CUL8R":"See You Later",
"CYA":"See You", "FAQ":"Frequently Asked Questions",
"FC":"Fingers Crossed", "FWIW":"For What It's Worth",
"FYI":"For Your Information", "GAL":"Get A Life",
"GG":"Good Game", "GN":"Good Night",
"GMTA":"Great Minds Think Alike", "GR8":"Great",
"G9":"Genius", "IC":"I See",
"ICQ":"I Seek you", "ILU":"I Love You",
"IMHO":"In My Honest Opinion", "IMO":"In My Opinion",
"IOW":"In Other Words", "IRL":"In Real Life",
"KISS":"Keep It Simple, Stupid",
"LDR":"Long Distance Relationship",
"LMAO":"Laugh My Ass Off",
"LOL":"Laughing Out Loud",
"LTNS":"Long Time No See",
"L8R":"Later",
"MTE":"My Thoughts Exactly",
"M8":"Mate",
"NRN":"No Reply Necessary",
"OIC":"Oh I See",
"PITA":"Pain In The Ass",
"PRT":"Party",
"PRW":"Parents Are Watching",
"QPSA":"Que Pasa",
"ROFL":"Rolling On The Floor Laughing",
"ROFLOL":"Rolling On The Floor Laughing Out Loud",
"ROTFLMAO":"Rolling On The Floor Laughing My A.. Off",
"SK8":"Skate",
"STATS":"Your sex and age",
"ASL":"Age, Sex, Location",
"THX":"Thank You",
"TTFN":"TaTa For Now",
"TTYL":"Talk To You Later",
"U":"You",
"U2":"You Too",
"U4E":"Yours For Ever",
"WB":"Welcome Back",
"WTF":"What The FUCK",
"WTG":"Way To Go",
"WUF":"Where Are You From",
"W8":"Wait",
"TFW":"That feeling when. TFW internet slang often goes in a caption to an image",
"MFW":"My face when",
"MRW":"My reaction when",
"IFYP":"I feel your pain",
"LOL":"Laughing out loud",
"TNTL":"Trying not to laugh",
"JK":"Just kidding",
"IDC":"I don't care",
"ILY":"I love you",
"IMU":"I miss you",
"ADIH":"Another day in hell",
"IDC":"I don't care",
"ZZZ":"Sleeping, bored, tired",
"WYWH":"Wish you were here",
"TIME":"Tears in my eyes",
"BAE":"Before anyone else",
"FIMH":"Forever in my heart",
"BSAAW":"Big smile and a wink",
"BWL":"Bursting with laughter",
"LMAO":"Laughing my ass off",
"BFF":"Best friends forever",
"CSL":"Can't stop laughing",
}

In [22]:
# function to remove the shortcut words

def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

### Punctuations Removal

In [23]:
# removing the punctuation in data

import string
string.punctuation


def remove_punctuations(text):
    text= "".join(char for char in text if char not in string.punctuation)
    return text

data['text']= data['text'].apply(remove_punctuations)

In [24]:
# removing the new line \n symbol in text

data['text']= data['text'].str.replace("\n"," ")

### StopWords Removal

In [25]:
#downloading english stopwords

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# function to remove the stopwords

StopWords=stopwords.words('english')

def stopwords_removal(text):
    text= text.lower()

    new_words=[]
    for word in text.split():
        if word not in StopWords:
            new_words.append(word)
        else:
            continue
    return " ".join(new_words)


In [27]:
data['text']= data['text'].apply(stopwords_removal)

### Emoji Removal

In [28]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 3.1 MB/s eta 0:00:00


In [29]:
import emoji

def emoji_removal(text):
    clean_text =emoji.demojize(text)
    return clean_text

In [30]:
data['text']= data['text'].apply(emoji_removal)


### Spell Corrections


In [31]:
# correcting the spelling in data

from textblob import TextBlob

def spell_corrections(text):
    txtblob=TextBlob(text)
    return txtblob.correct().string


In [32]:
data['text']= data['text'].apply(spell_corrections)

### Tokenization

In [33]:
#function to remove the numbers from text

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

data['text']= data['text'].apply(remove_numbers)

In [34]:
# function to tokenize words in the sentence

def word_tokenize(text):
    result=re.findall("[\w]+", text)
    return result

In [35]:
data['text'] = data['text'].apply(word_tokenize)

In [36]:
data.head(5)

,Index,text,label
0,106,"[real, good, moment, missssssssss, much]",0
1,217,"[reading, mania]",0
2,220,[comeagainjen],0
3,288,"[lascar, need, send, em, accountant, tomorrow,...",0
4,540,"[add, space, myspacecomlookthunder]",0


### Stemming and Lemmatization

In [37]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [38]:
# functon for stemming
from nltk.stem import PorterStemmer

def stemmer_func(text):
    stemmer=PorterStemmer()
    final= " ".join([stemmer.stem(word) for word in text])
    return final

# functon for Lemmatization
from nltk.stem import WordNetLemmatizer

def lemmatize_func(text):
  lemmatizer= WordNetLemmatizer()
  output=" ".join([lemmatizer.lemmatize(word) for word in text])
  return output

In [39]:
data['text']= data['text'].apply(lemmatize_func)

In [40]:
data['text'].head(5)

0                           real good moment miss much
1                                        reading mania
2                                         comeagainjen
3    lascar need send em accountant tomorrow oddly ...
4                      add space myspacecomlookthunder
Name: text, dtype: object

### Frequency Embeddings and Model trained on Naive Bayes


#### Bag Of Words(Counter Vectorizer)

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
BOW= CountVectorizer()
bag_of_vecors= BOW.fit_transform(data['text'])

In [44]:
sparse_BOW=bag_of_vecors.toarray()
counter=pd.DataFrame(sparse_BOW)

In [45]:
counter.shape

(10314, 15863)

#### TFIDF Vectorizer

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
tfidf= TfidfVectorizer()
tfidf_vectors= tfidf.fit_transform(data['text'])

In [48]:
sparse_pd=tfidf_vectors.toarray()
TFIDF= pd.DataFrame(sparse_pd)

In [49]:
TFIDF.shape

(10314, 15863)

#### Ngrams- Bigram

In [50]:
bigram=CountVectorizer(ngram_range=(2,2))
bigram_vectors=bigram.fit_transform(data["text"])

In [51]:
sparse=bigram_vectors.toarray()
BIGRAM= pd.DataFrame(sparse)

In [52]:
BIGRAM.shape

(10314, 64953)

#### Naive Bayes Model Training

In [53]:
# train test split the data

from sklearn.model_selection import train_test_split

x= TFIDF
y= data['label']

train_x, test_x, train_y, test_y= train_test_split(x,y, test_size=0.25)

In [54]:
# model training using naive bayes

from sklearn.naive_bayes import GaussianNB
nb_model= GaussianNB()

nb_model.fit(train_x, train_y)

GaussianNB()

In [55]:
# model predicton

pred_y= nb_model.predict(test_x)

In [56]:
# Model Metrics

from sklearn.metrics import accuracy_score, classification_report

print(accuracy_score(test_y, pred_y))

0.573090345094998


In [57]:
print(classification_report(test_y, pred_y))


              precision    recall  f1-score   support

           0       0.88      0.52      0.65      1996
           1       0.32      0.76      0.45       583

    accuracy                           0.57      2579
   macro avg       0.60      0.64      0.55      2579
weighted avg       0.75      0.57      0.61      2579

